In [1]:
# read data using pyspark
import pandas as pd
import plotly.express as px
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName("BD").getOrCreate()

In [2]:
# read data
app_df = spark.read.csv("../data/training.csv", header=True, inferSchema=True, nullValue="", sep=";")
app_df.limit(10).toPandas()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,music_genre
0,9.0,0.757000,0.0816,169627.0,0.0554,0.312000,A#,0.108,-19.406,Minor,0.0391,74.907,0.0703,Classical
1,43.0,0.996000,0.5850,230307.0,0.0140,0.929000,F,0.126,-29.092,Major,0.1020,127.593,0.2660,Classical
2,40.0,0.914000,0.2400,246733.0,0.3250,0.000398,G,0.115,-13.260,Major,0.0354,119.141,0.1090,Classical
3,52.0,0.700000,0.4620,225067.0,0.7410,0.000000,A#,0.340,-8.008,Minor,0.1620,NaN,0.5890,Hip-Hop
4,16.0,0.000008,0.3150,207987.0,0.9190,0.772000,C,0.342,-2.588,Major,0.0467,175.054,0.5740,Anime
5,32.0,0.041100,0.7190,190312.0,0.9170,0.000645,A,0.226,-3.400,Minor,0.0441,NaN,0.4910,Electronic
6,40.0,0.721000,0.6780,323120.0,0.7810,0.477000,F,0.106,-8.721,Minor,0.0496,82.762,0.5300,Jazz
7,37.0,0.028200,0.5150,221378.0,0.8750,0.418000,A,0.218,-5.708,Minor,0.0364,165.991,0.4970,Anime
8,24.0,0.016300,0.6780,255480.0,0.8590,0.000000,G#,0.330,-4.464,Major,0.0415,131.027,0.6600,Anime
9,43.0,0.223000,0.5390,230107.0,0.8200,0.000001,A,0.154,-3.913,Major,0.0363,104.033,0.5600,Country


In [3]:
from pyspark.sql.functions import count, when, isnan, col

# Analyze data
print("Number of rows: ", app_df.count())
numerical_features = [t[0] for t in app_df.dtypes if t[1] == 'int' or t[1] == 'double']
categorical_features = [t[0] for t in app_df.dtypes if t[1] == 'string']
print("Number of numerical features: ", len(numerical_features))
print("Number of categorical features: ", len(categorical_features))

# print statistical summary of numerical features
descriptors = app_df.select(numerical_features).describe().toPandas()
# add missing values count
missing_values = app_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in numerical_features]).toPandas()
missing_values["summary"] = "missing"
descriptors = pd.concat([descriptors, missing_values], ignore_index=True)
descriptors = descriptors.transpose()
descriptors = descriptors.rename(columns=descriptors.iloc[0]).drop(descriptors.index[0])
descriptors = descriptors.astype(float)

print("Statistical summary of numerical features:")
display(descriptors.round(2))

# print statistical summary of categorical features
print("Statistical summary of categorical features:")
for col in categorical_features:
    tmp_df = app_df.groupBy(col).count().toPandas().sort_values('count', ascending=False)
    tmp_df['percentage'] = tmp_df['count'] / tmp_df['count'].sum() * 100
    # use plotlyexpress to plot the data
    fig = px.bar(tmp_df, x=col, y='count', title=f"{col} ({app_df.select(col).distinct().count()})")
    fig.show()
    tmp_df = tmp_df.transpose().round(2)
    display(tmp_df.rename(columns=tmp_df.iloc[0]).drop(tmp_df.index[0]))

Number of rows:  30003
Number of numerical features:  11
Number of categorical features:  3
Statistical summary of numerical features:


,count,mean,stddev,min,max,missing
popularity,30002.0,44.22,15.56,0.00,97.00,1.0
acousticness,30002.0,0.31,0.34,0.00,1.00,1.0
danceability,30002.0,0.56,0.18,0.06,0.98,1.0
duration_ms,30002.0,220503.48,126619.91,-1.00,4276000.00,1.0
energy,30002.0,0.60,0.27,0.00,1.00,1.0
instrumentalness,30002.0,0.18,0.33,0.00,0.99,1.0
liveness,30002.0,0.19,0.16,0.01,0.99,1.0
loudness,30002.0,-9.18,6.23,-47.05,3.74,1.0
speechiness,30002.0,0.09,0.10,0.02,0.94,1.0
tempo,27016.0,120.00,30.68,34.47,220.28,2987.0


Statistical summary of categorical features:


,G,C,C#,D,A,F,B,E,G#,A#,F#,D#,None
count,3486,3270,3208,3192,2934,2599,2255,2226,2020,1999,1876,937,1
percentage,11.618838,10.89891,10.692264,10.638936,9.779022,8.662467,7.515915,7.419258,6.73266,6.662667,6.252708,3.123021,0.003333


,Major,Minor,None
count,19212,10790,1
percentage,64.033597,35.96307,0.003333


,Jazz,Hip-Hop,Classical,Country,Blues,Rap,Alternative,Electronic,Rock,Anime,None
count,3048,3025,3023,3021,3010,3001,3000,2971,2958,2945,1
percentage,10.158984,10.082325,10.075659,10.068993,10.03233,10.002333,9.999,9.902343,9.859014,9.815685,0.003333


In [4]:
# Basic transformations
# rellenamos todos los tempo vacios con 0
app_df = app_df.fillna(0, subset=["tempo"])
# Hay una fila (29617) que todos los valores son nulos, nos la cargamos
app_df = app_df.dropna()
print("Number of rows after dropping rows with null values: ", app_df.count())
# como se puede ver solo hemos perdido una fila

Number of rows after dropping rows with null values:  30002


In [19]:
# Split data in dev and test
dev_df, test_df = app_df.randomSplit([0.9, 0.1], seed=12345)
print("Number of rows in dev data: ", dev_df.count())
print("Number of rows in test data: ", test_df.count())
print("Intersecting rows: ", dev_df.intersect(test_df).count())

Number of rows in dev data:  26898
Number of rows in test data:  3104
Intersecting rows:  0


In [8]:
# naive approach to clasification (baseline)
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline

# preprocessing
indexer = StringIndexer(inputCols=categorical_features, outputCols=[col + "_index" for col in categorical_features])
ohe = OneHotEncoder(inputCols=[col + "_index" for col in categorical_features], outputCols=[col + "_ohe" for col in categorical_features])
assembler = VectorAssembler(inputCols=[col + "_ohe" for col in categorical_features] + numerical_features, outputCol="features")

In [9]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier

# cross validation
estimator = RandomForestClassifier(labelCol="music_genre_index", featuresCol="features")
estimator_params = ParamGridBuilder().addGrid(RandomForestClassifier.numTrees, [10, 20, 30]).build()
evaluator = MulticlassClassificationEvaluator(labelCol="music_genre_index", predictionCol="prediction", metricName="f1")
cross = CrossValidator(estimator=estimator, estimatorParamMaps=estimator_params, evaluator=evaluator, numFolds=10)

# build the pipeline
pipeline = Pipeline(stages=[indexer, ohe, assembler, cross])
model = pipeline.fit(dev_df)

# evaluate the model
predictions = model.transform(test_df)
f1 = evaluator.evaluate(predictions)
print("F1 score: ", f1)

F1 score:  0.9990326643757047


In [16]:
dev_df.show()

+----------+------------+------------------+-----------+------------------+--------------------+---+--------+------------------+-----+-----------+------------------+------------------+-----------+
|popularity|acousticness|      danceability|duration_ms|            energy|    instrumentalness|key|liveness|          loudness| mode|speechiness|             tempo|           valence|music_genre|
+----------+------------+------------------+-----------+------------------+--------------------+---+--------+------------------+-----+-----------+------------------+------------------+-----------+
|       0.0|     8.96E-5|             0.669|   183907.0|             0.904|  0.6990000000000001| C#|  0.0544|            -3.485|Major|     0.0705|           128.011|             0.303| Electronic|
|       0.0|     3.47E-4|             0.214|   355540.0|             0.969|               0.246|  B|   0.138|            -3.924|Major|      0.106|159.94799999999998|            0.0369|Alternative|
|       0.0|   